# libraries 

In [13]:
import requests #The requests library for HTTP requests in Python
import csv
import pandas as pd
from secrets import IEX_CLOUD_API_TOKEN_LIVE
from secrets import ALPHA_API_KEY
from symbols import symbols


# Loop through the tickers and bring up the daily OHLC

In [14]:
gappedStocks = []

# difference between two numbers
def changePercent(first, second):
    first = round(first, 2)
    second = round(second, 2)
    return ((second - first) / first) * 100

for i in symbols:
    symbol = i
    
    api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={ALPHA_API_KEY}'

    r = requests.get(api_url)
    data = r.json()

    for i in data:
        if 'Time Series (Daily)' in data:
            for dt, values in data['Time Series (Daily)'].items(): 

                # get the values from the API dictionary
                open = float(values['1. open'])
                high = float(values['2. high'])
                low = float(values['3. low'])
                close = float(values['4. close'])
                volume = float(values['6. volume'])
                gap = changePercent(close, high)
                
                if gap > 15 and volume > 10000000:
                    gappedObj = {
                        'symbol': symbol,
                        'date': dt,
                        'gap (%)': gap,
                        'open': open,
                        'high': high,
                        'low': low,
                        'close': close,
                        'volume': volume
                    }
                    
                    gappedStocks.append(gappedObj)


print(gappedStocks)

   



[{'symbol': 'WBAI', 'date': '2021-09-03', 'gap (%)': 21.67774086378739, 'open': 12.44, 'high': 14.65, 'low': 10.86, 'close': 12.04, 'volume': 37290857.0}, {'symbol': 'WBAI', 'date': '2021-09-03', 'gap (%)': 21.67774086378739, 'open': 12.44, 'high': 14.65, 'low': 10.86, 'close': 12.04, 'volume': 37290857.0}, {'symbol': 'AMC', 'date': '2021-12-13', 'gap (%)': 18.889845094664377, 'open': 27.49, 'high': 27.63, 'low': 22.46, 'close': 23.24, 'volume': 84199780.0}, {'symbol': 'AMC', 'date': '2021-12-01', 'gap (%)': 22.296114805740277, 'open': 34.71, 'high': 34.9362, 'low': 26.85, 'close': 28.57, 'volume': 63510555.0}, {'symbol': 'AMC', 'date': '2021-12-13', 'gap (%)': 18.889845094664377, 'open': 27.49, 'high': 27.63, 'low': 22.46, 'close': 23.24, 'volume': 84199780.0}, {'symbol': 'AMC', 'date': '2021-12-01', 'gap (%)': 22.296114805740277, 'open': 34.71, 'high': 34.9362, 'low': 26.85, 'close': 28.57, 'volume': 63510555.0}, {'symbol': 'BEST', 'date': '2021-11-01', 'gap (%)': 19.84732824427481, 